In [23]:
from pymongo import MongoClient
import json
from os import path
import pandas as pd

CONNECTION_STRING = 'mongodb://localhost:27017'
DATABASE = 'quiz-book'
COLLECTION_ENGLISH_WORDS = 'english_words_v1'


client = MongoClient(CONNECTION_STRING)
db = client[DATABASE]


def load_filepaths():
    source_index = '../public/assets/english/words/index.json'

    filenames = []

    with open(source_index, 'r') as f:
        filenames = json.load(f)

    pathroot = path.dirname(source_index)
    return [path.join(pathroot, fname) for fname in filenames]


def load_words(filepath):
    
    df = pd.read_csv(filepath)
    words = df['english'].tolist()

    words_cursor = db[COLLECTION_ENGLISH_WORDS].find({"en": {"$in": words}}, {'_id': 0})
    
    words_in_dict = [r for r in words_cursor]
    # print(words_in_dict)

    df_copy = df.copy()
    for i, row in df_copy.iterrows():
        en = row['english']
        current_record = next(filter(lambda r: r['en']==en, words_in_dict), None)
        if current_record is not None:
            df_copy.at[i, 'symbol'] = current_record['phonetic_us']
            if current_record['zh'] is not None and current_record['zh'] != '':
                df_copy.at[i, 'chinese'] = current_record['zh']

    df_copy.to_csv(filepath, index=False)



def test():
    client = MongoClient(CONNECTION_STRING)
    english_words_v1 = client[DATABASE][COLLECTION_ENGLISH_WORDS]
    cursor = english_words_v1.find({'en': 'a'})
    arr = [i for i in cursor]
    print(arr)


def run():
    filepaths = load_filepaths()
    for fp in filepaths:
        load_words(fp)

    # test()


run()









In [15]:
a = [0, 1]
next(filter(lambda s: s == 2, a), None)